In [1]:
import random
import pandas as pd
import numpy

In [2]:
def createchrom(x, y, numberofchrom):
    ######### xypair is a 16 bit chromosome represented as (x,y). Where x is the index of x chrom and y in the index of y chrom  
    ######### say, (0,0) represents first x and first y chrom. There are numberofchrom*numberofchrom such (x,y) pairs ### 
    xypair = [[0 for i in range(2)] for j in range(numberofchrom*numberofchrom)]
    i = 0
    for j in range(numberofchrom):
        for k in range(numberofchrom):
            xypair[i][0] = j
            xypair[i][1] = k
            i = i+1
    return xypair    

# The polynomial used is z = (4-2.1(x^2)+(x^4)/3)* x^2 + xy + (-4+4(y^2))* y^2

In [3]:
def Evaluate(populationx, populationy, xypair, numberofchrom):
    xvalues = [0]*numberofchrom
    yvalues = [0]*numberofchrom
    ###  evaluate the value of x chromosome ####
    for j in range(numberofchrom):
        chrom = populationx[j]
        chrom = chrom[::-1]
        v = 0
        for i in range(8):
            v = v+(chrom[i]*pow(2,i))
        v = v*0.039-5
        xvalues[j] = v
    ### evaluate the value of y chromosome ###
    for j in range(numberofchrom):
        chrom = populationy[j]
        chrom = chrom[::-1]
        v = 0
        for i in range(8):
            v = v+(chrom[i]*pow(2,i))
        v = v*0.039-5
        yvalues[j] = v
    
    ####  zvalues are calculated for each (x,y) pair  ###
    zvalues = [0]*numberofchrom*numberofchrom
    numpairs = numberofchrom*numberofchrom
    for i in range(numpairs):
        xc = xypair[i][0]
        yc = xypair[i][1]
        xval = xvalues[xc]
        yval = yvalues[yc]
        zvalues[i] = (4-2.1*pow(xval,2)+pow(xval,4)/3)*pow(xval, 2)+ xval*yval + (-4+4*pow(yval,2))*pow(yval,2)
        
    return zvalues
    


In [4]:
def Selection(populationx, populationy, xypair, numberofchrom, zvalues):
    newpop_xy = [[0 for i in range(2)] for j in range(numberofchrom*numberofchrom)]
    xyp = pd.DataFrame(xypair)
    zv = pd.DataFrame(zvalues)
    cbind = pd.concat([xyp, zv], axis=1, ignore_index=True)
    cbind.rename(columns={0 : 'xindex', 1 : 'yindex', 2 : 'zvalue'},inplace=True)
    sorted_df = cbind.sort_values(by='zvalue')
    ###  Now we construct the roullette wheel. the area the chromosome get is 
    ###  in the reverse order of (zvalue/total zvalue of all chromosome) as we are minimizing. If we are maximizing
    ###  the area can be in probability (zvalue/total zvalue of all chromosome)
    numpairs = numberofchrom*numberofchrom
    totzval = 0
    for i in range(numpairs):
        totzval = totzval+zvalues[i]

    area_proportion = [0]*numpairs
    sortz = list(sorted_df['zvalue'])

    for i in range(numpairs):
        area_proportion[i] = (sortz[i]/totzval)

    area_proportion = area_proportion[::-1]

    totarea = 0
    for i in range(numpairs):
        totarea = totarea+area_proportion[i]
    
    ### bins[low,high] has the values in which if the randomly seclected value lie then that chrosome is selected
    bins = [[0 for i in range(2)] for j in range(numpairs)]
    bins[0][0] = 0
    bins[0][1] = area_proportion[0]
    for i in range(1, numpairs):
        bins[i][0] = bins[i-1][1]
        bins[i][1] = bins[i][0] + area_proportion[i]

    ### Now rotate the roullette wheel numpairs times to create a new set of population 
    sortxc = list(sorted_df['xindex'])
    sortyc = list(sorted_df['yindex'])

    sortpair = [[0 for i in range(2)] for j in range(numberofchrom*numberofchrom)]
    for i in range(numberofchrom*numberofchrom):
        sortpair[i][0] = sortxc[i]
        sortpair[i][1] = sortyc[i] 
    prob = [0]*numpairs
    for i in range(numpairs):
        prob[i] = random.random()
        for j in range(numpairs):
            if(prob[i]> bins[j][0] and prob[i] <=bins[j][1]):
                newpop_xy[i][0] = sortpair[j][0]
                newpop_xy[i][1] = sortpair[j][1]
            
    newzval = [0]*numberofchrom*numberofchrom
    newzval = Evaluate(populationx, populationy, newpop_xy, numberofchrom)
                
    ######  Now we again sort the newpop based on their fitness
    xyp = pd.DataFrame(newpop_xy)
    zv = pd.DataFrame(newzval)
    cbind = pd.concat([xyp, zv], axis=1, ignore_index=True)
    cbind.rename(columns={0 : 'xindex', 1 : 'yindex', 2 : 'zvalue'},inplace=True)
    sorted_df2 = cbind.sort_values(by='zvalue')
    
    sortxc = list(sorted_df['xindex'])
    sortyc = list(sorted_df['yindex'])
    sortz = list(sorted_df['zvalue'])

    sortpair = [[0 for i in range(2)] for j in range(numberofchrom*numberofchrom)]
    for i in range(numberofchrom*numberofchrom):
        sortpair[i][0] = sortxc[i]
        sortpair[i][1] = sortyc[i]
        
    return sortpair, sortz

In [5]:
def Crossover(populationx, populationy, numberofchrom):
    oldx = [[0 for i in range(8)] for j in range(numberofchrom)]
    oldy = [[0 for i in range(8)] for j in range(numberofchrom)] 
    oldx = populationx
    oldy = populationy
    newx = [[0 for i in range(8)] for j in range(numberofchrom)]
    newy = [[0 for i in range(8)] for j in range(numberofchrom)]

    l = 0
    for i in range(numberofchrom -1):
        px1 = oldx[i]
        px2 = oldx[i+1]
        crosspoint = random.randint(1,7)
        child1 = [0]*8
        child2 = [0]*8
    
        for j in range(crosspoint):
            child1[j] = px1[j]
            child2[j] = px2[j]
        for k in range(crosspoint, 8):
            child1[k] = px2[k]
            child2[k] = px1[k]
        
        for l in range(8):
            newx[i][l] = child1[l]
            newx[i+1][l] = child2[l]
    
    l = 0
    for i in range(numberofchrom -1):
        py1 = oldy[i]
        py2 = oldy[i+1]
        crosspoint = random.randint(1,7)
        child1 = [0]*8
        child2 = [0]*8
    
        for j in range(crosspoint):
            child1[j] = py1[j]
            child2[j] = py2[j]
        for k in range(crosspoint, 8):
            child1[k] = py2[k]
            child2[k] = py1[k]
        
        for l in range(8):
            newy[i][l] = child1[l]
            newy[i+1][l] = child2[l]
    
    return newx, newy

In [6]:
def Mutation(populationx, populationy, numberofchrom):
    oldx = [[0 for i in range(8)] for j in range(numberofchrom)]
    oldy = [[0 for i in range(8)] for j in range(numberofchrom)] 
    oldx = populationx
    oldy = populationy
    newx = [[0 for i in range(8)] for j in range(numberofchrom)]
    newy = [[0 for i in range(8)] for j in range(numberofchrom)]
    
    for i in range(numberofchrom):
        for j in range(8):
            newx[i][j] = oldx[i][j]
    for i in range(numberofchrom):
        p = oldx[i]
        mutatpoint = random.randint(1,7)
        if(oldx[i][mutatpoint] == 0):
            newx[i][mutatpoint] = 1
        else:
            newx[i][mutatpoint] = 0
            
    for i in range(numberofchrom):
        for j in range(8):
            newy[i][j] = oldy[i][j]
    for i in range(numberofchrom):
        p = oldy[i]
        mutatpoint = random.randint(1,7)
        if(oldy[i][mutatpoint] == 0):
            newy[i][mutatpoint] = 1
        else:
            newy[i][mutatpoint] = 0
            
    return newx, newy


In [7]:
numberofchrom = 5
#### population = 8 bit representation * number of chromosomes 
populationx = [[0 for i in range(8)] for j in range(numberofchrom)]
populationy = [[0 for i in range(8)] for j in range(numberofchrom)]

for j in range(numberofchrom):
    for i in range(8):
        t = random.random()
        if(t>=0.5):
            populationx[j][i] = 0
        else:
            populationx[j][i] = 1
            
for j in range(numberofchrom):
    for i in range(8):
        t = random.random()
        if(t>=0.5):
            populationy[j][i] = 0
        else:
            populationy[j][i] = 1
            
xypair = createchrom(populationx, populationy, numberofchrom)
zvalues = Evaluate(populationx, populationy, xypair, numberofchrom)

sortpair, newzv = Selection(populationx, populationy, xypair, numberofchrom, zvalues)
minobj = newzv[0]
xoptindex = sortpair[0][0]
yoptindex = sortpair[0][1]

itera = 0
while (itera<100):
    newx, newy = Crossover(populationx, populationy, numberofchrom)
    #### 20% time do mutation ####
    tutul = random.randint(1,10)
    if tutul>8:
        newx, newy = Mutation(newx, newy, numberofchrom)
    newz = Evaluate(newx, newy, sortpair, numberofchrom)

    sortpair1, newzv1 = Selection(newx, newy, sortpair, numberofchrom, newz)

    if(newzv1[0]<minobj):
        minobj = newzv1[0]
        xoptindex = sortpair1[0][0]
        yoptindex = sortpair1[0][1]
        pox = [[0 for i in range(8)] for j in range(numberofchrom)]
        poy = [[0 for i in range(8)] for j in range(numberofchrom)]
        for j in range(numberofchrom):
            for i in range(8):
                pox[j][i] = newx[j][i]
        for j in range(numberofchrom):
            for i in range(8):
                poy[j][i] = newy[j][i]

    itera = itera +1
    
    
chrom = pox[xoptindex]
chrom = chrom[::-1]
v = 0
for i in range(8):
    v = v+(chrom[i]*pow(2,i))
v = v*0.039-5
xval = v

chrom = poy[yoptindex]
chrom = chrom[::-1]
v = 0
for i in range(8):
    v = v+(chrom[i]*pow(2,i))
v = v*0.039-5
yval = v


print("The value of x:", xval)
print("The value of y:", yval)
print("The optimal objective is", minobj)

The value of x: -0.008000000000000007
The value of y: 0.7720000000000002
The optimal objective is -0.9690682955775124
